# 1. 필요 모듈 불러오기

In [1]:
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import folium
import branca
import branca.colormap as cm
from folium import plugins
from folium.plugins import HeatMap
import geopandas as gpd
import seaborn as sns
import random
import os

pd.set_option('display.max_columns', None)

a = os.getcwd()
os.chdir(a)
a

'/opt/app-root/src/승순/Full table'

# 2. 필요 데이터 로드

동별 인구데이터 불러오기

In [2]:
data1 = pd.read_csv('대전시 행정동별 인구통계(2018).csv')
data1.head()

,Unnamed: 0,구,동,총인구수,청소년,생산가능 인구,고령인구
0,0,대덕구,대화동,7877.0,1491.0,5776.0,1444.0
1,1,대덕구,덕암동,14715.0,3707.0,10985.0,2001.0
2,2,대덕구,목상동,7005.0,1871.0,5396.0,652.0
3,3,대덕구,법1동,10362.0,2384.0,7403.0,1953.0
4,4,대덕구,법2동,17584.0,4612.0,13224.0,2385.0


동별 격자 데이터 불러오기

In [3]:
filename = "행정동_격자.json"
file = open(filename)
data2 = gpd.read_file(file)
data2.head(10)

,gid,cars_cnt,행정동,geometry
0,다마846996,0,기성동,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,0,기성동,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,다마847996,0,벌곡면,"MULTIPOLYGON (((127.32982 36.19342, 127.32982 ..."
3,다마847997,0,벌곡면,"MULTIPOLYGON (((127.32982 36.19432, 127.32982 ..."
4,다마847998,0,벌곡면,"MULTIPOLYGON (((127.32982 36.19522, 127.32982 ..."
5,다마848985,0,진산면,"MULTIPOLYGON (((127.33096 36.18350, 127.33096 ..."
6,다마848986,0,벌곡면,"MULTIPOLYGON (((127.33096 36.18440, 127.33095 ..."
7,다마848987,0,벌곡면,"MULTIPOLYGON (((127.33095 36.18530, 127.33095 ..."
8,다마848988,0,벌곡면,"MULTIPOLYGON (((127.33095 36.18621, 127.33095 ..."
9,다마848989,0,벌곡면,"MULTIPOLYGON (((127.33095 36.18711, 127.33095 ..."


행정동 데이터의 경우 '면' 데이터도 있어서 개수에 차이가 발생

In [4]:
print ( len(data1['동'].unique() ) )
print ( len(data2['행정동'].unique() ) )

79
90


행정동 컬럼에서 면이 있는 행 삭제

In [5]:
a = data2[~data2.행정동.str.contains("면")] #
#len(a['행정동'].unique())
a.head()

,gid,cars_cnt,행정동,geometry
0,다마846996,0,기성동,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,0,기성동,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
16,다마848996,0,기성동,"MULTIPOLYGON (((127.33094 36.19342, 127.33093 ..."
17,다마848997,0,기성동,"MULTIPOLYGON (((127.33093 36.19432, 127.33093 ..."
18,다마848998,0,기성동,"MULTIPOLYGON (((127.33093 36.19522, 127.33093 ..."


병합을 하기 위해 '행정동'을 '동'으로 컬럼명 변경

In [6]:
a.rename(columns = {'행정동':'동'}, inplace=True)
a.head()

,gid,cars_cnt,동,geometry
0,다마846996,0,기성동,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,다마847995,0,기성동,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
16,다마848996,0,기성동,"MULTIPOLYGON (((127.33094 36.19342, 127.33093 ..."
17,다마848997,0,기성동,"MULTIPOLYGON (((127.33093 36.19432, 127.33093 ..."
18,다마848998,0,기성동,"MULTIPOLYGON (((127.33093 36.19522, 127.33093 ..."


편의를 위해 컬럼순서 변경

In [7]:
#merged_df = pd.merge(df1, df2, left_on = "employee", right_on = "name")
b = pd.merge(a, data1, on=["동"])
b = b[['구','동','총인구수','청소년','생산가능 인구','고령인구','cars_cnt','gid','geometry']]
b.head()

,구,동,총인구수,청소년,생산가능 인구,고령인구,cars_cnt,gid,geometry
0,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마846996,"MULTIPOLYGON (((127.32871 36.19342, 127.32871 ..."
1,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마847995,"MULTIPOLYGON (((127.32983 36.19252, 127.32982 ..."
2,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848996,"MULTIPOLYGON (((127.33094 36.19342, 127.33093 ..."
3,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848997,"MULTIPOLYGON (((127.33093 36.19432, 127.33093 ..."
4,서구,기성동,3990.0,480.0,2385.0,1382.0,0,다마848998,"MULTIPOLYGON (((127.33093 36.19522, 127.33093 ..."


In [8]:
import folium

converted_json = b.to_json()
m = folium.Map(
    location=[36.34156,127.39704],
    zoom_start=11
)

folium.GeoJson(
    converted_json,
).add_to(m)

m

In [9]:
b1=b

In [10]:
#b.to_file("대전시 행정구역별 인구 with geometry.json", driver="GeoJSON")

결과물 파일로 저장

In [11]:
#b.to_csv('대전시 행정구역별 인구 with geometry.csv')